# Verificação das hipóteses relacionadas aos feedbacks

## Verificação das hipóteses 2, 4 e 10
Hipótese 2: Os clientes que cancelaram os pedidos não respondem o questionário de satisfação \
Hipótese 4: Se o pedido atrasar sua nota será menor que três \
Hipótese 10: A taxa de respondidos entre os cancelados é menor que a taxa de respondidos no geral 

## Definição do datafreme

In [1]:
from pyspark.sql import SparkSession, functions as F
spark = SparkSession.builder.getOrCreate()
df_reviews = spark.read \
                 .option('escape', '\"') \
                 .csv('./dataset/olist_order_reviews_dataset.csv', header=True, multiLine=True)
df_orders = spark.read \
                 .option('escape', '\"') \
                 .csv('./dataset/olist_orders_dataset.csv', header=True, multiLine=True)
df = df_orders.join(df_reviews, df_orders.order_id == df_reviews.order_id)

In [2]:
df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- order_delivered_carrier_date: string (nullable = true)
 |-- order_delivered_customer_date: string (nullable = true)
 |-- order_estimated_delivery_date: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)



### Criação dataframe com pedidos cancelados

In [3]:
df_canceled = df.filter(F.col('order_status')=='canceled')

### Verificação dos valores dos feedbacks dos pedidos cancelados 
### Verifica a hipótese 2

In [4]:
df_canceled.select('review_comment_message').show(5, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------+
|review_comment_message                                                                                                       |
+-----------------------------------------------------------------------------------------------------------------------------+
|Péssimo                                                                                                                      |
|Pedi reembolso e sem resposta até momento                                                                                    |
|null                                                                                                                         |
|Espero receber esta semana, o que não seria tão extravagante, porém...mais atrazo do que isso me faria desistir do baratheon.|
|null                                                                                                   

### Verificação das notas em pedidos cancelados 
#### Verifica hipótese 4

In [5]:
print("O número de pedidos cancelados com nota >= 3 é de" 
      ,df_canceled.filter(F.col('review_score')>=3).count(),"pedidos")

O número de pedidos cancelados com nota >= 3 é de 148 pedidos


### Calculo da taxa de feedbacks respondidos dentre os pedidos cancelados

In [6]:
num_canceled_answered = df_canceled.filter(df_canceled.review_comment_message.isNotNull()).count()
num_canceled = df_canceled.count()
percent_canceladed_answered = num_canceled_answered / num_canceled * 100
print("Taxa de cancelados respondidos =",round(percent_canceladed_answered, 2))

Taxa de cancelados respondidos = 67.73


### Calculo da taxa de feedbacks respondidos dentre todos os pedidos

In [7]:
df = df_reviews
df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)



In [8]:
df_with_comment = df.filter(df.review_comment_message.isNotNull())

In [9]:
percent_answered = df_with_comment.count() / df.count() * 100;
print("Taxa de respondidos =",round(percent_answered, 2))

Taxa de respondidos = 41.75


## Conclusões

### A hipótese 2 é inválida pois as pessoas que cancelaram os pedidos foram capazes de responder o feedback, visto as mensagens de feedback exitentes entre os pedidos cancelados

### A hipótese 4 é inválida pois existem pedidos cancelados que possuem nota maior ou igual a 3, são 148 pedidos com essas características

### A hipótese 10 é inválida pois a porcentagem de feedbacks respondidos  dentro os pedidos cancelados foi maior que a porcentagem dentre todos os pedidos. A taxa de respondidos dentre os cancelados é de 67.73 % e a taxa de respondidos no total é de 41.75 %